In [49]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
!pip install Textblob

In [51]:
from textblob import TextBlob

In [52]:
import pandas as pd

In [53]:
data_path = '/content/drive/MyDrive/Portfolio/Sentiment Analysis User Reviews App GetContact/ProssedGetContact.csv'

In [ ]:
!pip3 install googletrans==3.1.0a0

In [ ]:
# Importing library yang dibutuhkan
import googletrans
from googletrans import Translator

In [ ]:
data = pd.read_csv(data_path)
data.head()

In [42]:
translator = Translator()
translations = {}
for column in data.columns:
    # Unique elements of the column
    unique_elements = data[column].unique()
    for element in unique_elements:
        # Adding all the translations to a dictionary (translations)
        translations[element] = translator.translate(element).text
translations

{'nomor gbsa verifikasi udh puluhan kali dicoba': "The number can't be verified, I've tried dozens of times",
 'some number can refuse to be seen on their tag ussually a scammer': 'some number can refuse to be seen on their tag ussually a scammer',
 'kembalikan woyy': 'return woyy',
 nan: 'in',
 'gratis skrg berbayar uninstall aja': "it's free now it's paid to just uninstall",
 'buruk': 'bad',
 'mantap': 'solid',
 'kebuka aplikasinya coba keterangannya mengirim permintaan verifikasi gaada verifikasi': 'Open the application, try the description, send a verification request, there is no verification',
 'kesini apknya kesana': 'the apk goes here',
 'menyesal langganan langsung motong pulsa kasih bintang ribu langganan nggak tahunya pulsa potong percaya app': "regret the subscription, immediately cut credit, give thousands of stars, subscribe, don't know, credit cut, believe the app",
 'syamsul': 'syamsul',
 'feature jelek yg download uninstall aja': 'Bad features that download just uninst

In [43]:
data.replace(translations, inplace = True)
data

,content_normalisasi2
0,"The number can't be verified, I've tried dozen..."
1,some number can refuse to be seen on their tag...
2,return woyy
3,in
4,it's free now it's paid to just uninstall
...,...
4378,I've tried to log in completely by filling in ...
4379,"it's still good, but it's a premium, if there'..."
4380,Just give me a star to see the results
4381,You're stingy just looking at people's premium...


In [44]:
data.to_excel('/content/drive/MyDrive/Portfolio/Sentiment Analysis User Reviews App GetContact/LabellingData GetContact/GetContact_DataTranslated.xlsx', index=False)

In [45]:
#SPLIT DATA
import pandas as pd

dataset = pd.read_excel('/content/drive/MyDrive/Portfolio/Sentiment Analysis User Reviews App GetContact/LabellingData GetContact/GetContact_DataTranslated.xlsx')

# Hitung jumlah baris dalam dataset
num_rows = len(dataset)

# Hitung jumlah baris yang akan digunakan sebagai pelatihan (disini 50%)
train_size = int(0.5 * num_rows)

# Bagi dataset menjadi subset pelatihan dan pengujian
train_data = dataset[:train_size]
test_data = dataset[train_size:]

# Simpan subset pelatihan ke dalam file xlsx
train_data.to_excel('/content/drive/MyDrive/Portfolio/Sentiment Analysis User Reviews App GetContact/LabellingData GetContact/GetContact_DataTrainEnglish.xlsx', index=False)

# Simpan subset pengujian ke dalam file xlsx
test_data.to_excel('/content/drive/MyDrive/Portfolio/Sentiment Analysis User Reviews App GetContact/LabellingData GetContact/GetContact_DataTestEnglish.xlsx', index=False)


In [46]:
!pip install Textblob

In [47]:
from textblob import TextBlob

In [48]:
train_data

,content_normalisasi2
0,"The number can't be verified, I've tried dozen..."
1,some number can refuse to be seen on their tag...
2,return woyy
3,in
4,it's free now it's paid to just uninstall
...,...
2186,keep buying
2187,why pay?
2188,"Can't see the tag now, just try to limit it by..."
2189,"I really don't like it, why do you pay like th..."


In [54]:
data_content = list(train_data['content_normalisasi2'])
polaritas = 0

status = []
total_positif = total_negatif = total = 0

for i, tweet in enumerate(data_content):
    analysis = TextBlob(tweet)
    polaritas += analysis.polarity

    if analysis.sentiment.polarity > 0.0:
        total_positif += 1
        status.append('Positif')
    else:
        total_negatif += 1
        status.append('Negatif')

    total += 1

print(f'Hasil Analisis Data:\nPositif = {total_positif}\nNegatif = {total_negatif}')
print(f'\nTotal Data : {total}')

Hasil Analisis Data:
Positif = 600
Negatif = 1591

Total Data : 2191


In [55]:
status = pd.DataFrame({'Label': status})
train_data['Label'] = status
train_data

<ipython-input-55-ddb1f5b9bf00>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Label'] = status


,content_normalisasi2,Label
0,"The number can't be verified, I've tried dozen...",Negatif
1,some number can refuse to be seen on their tag...,Negatif
2,return woyy,Negatif
3,in,Negatif
4,it's free now it's paid to just uninstall,Positif
...,...,...
2186,keep buying,Negatif
2187,why pay?,Negatif
2188,"Can't see the tag now, just try to limit it by...",Positif
2189,"I really don't like it, why do you pay like th...",Negatif


In [56]:
# Memisahkan data berdasarkan label
positif_data = train_data[train_data['Label'] == 'Positif']
negatif_data = train_data[train_data['Label'] == 'Negatif']

In [57]:

# Menentukan jumlah sampel minimum di 2 kelas kelas
min_samples = min(total_positif, total_negatif)

In [58]:
from sklearn.utils import resample

In [59]:
# Melakukan penyeimbangan data dengan undersampling
positif_data_resampled = resample(positif_data, replace=False, n_samples=min_samples, random_state=42)
negatif_data_resampled = resample(negatif_data, replace=False, n_samples=min_samples, random_state=42)

In [60]:
# Menggabungkan kembali data yang telah disusun ulang
balanced_data = pd.concat([positif_data_resampled, negatif_data_resampled])

In [62]:
data_content_balanced = list(balanced_data['content_normalisasi2'])
polaritas_balanced = 0

status_balanced = []
total_positif_balanced = total_negatif_balanced = total_balanced = 0

for i, tweet in enumerate(data_content_balanced):
    analysis = TextBlob(tweet)
    polaritas_balanced += analysis.polarity

    if analysis.sentiment.polarity > 0.0:
        total_positif_balanced += 1
        status_balanced.append('Positif')
    else:
        total_negatif_balanced += 1
        status_balanced.append('Negatif')

    total_balanced += 1

print(f'Hasil Analisis Data Setelah Penyeimbangan:\nPositif = {total_positif_balanced}\nNegatif = {total_negatif_balanced}')
print(f'\nTotal Data Setelah Penyeimbangan: {total_balanced}')

Hasil Analisis Data Setelah Penyeimbangan:
Positif = 600
Negatif = 600

Total Data Setelah Penyeimbangan: 1200


In [63]:
train_data.to_excel('/content/drive/MyDrive/Portfolio/Sentiment Analysis User Reviews App GetContact/LabellingData GetContact/GetContact_DataTrainEngLabeled.xlsx',index=False)

In [64]:
balanced_data.to_excel('/content/drive/MyDrive/Portfolio/Sentiment Analysis User Reviews App GetContact/LabellingData GetContact/GetContact_DataTrainEngLabeledBalanced.xlsx', index=False)

In [66]:
import pandas as pd
from googletrans import Translator

def translate_to_indonesian(text):
    translator = Translator()
    result = translator.translate(text, src='en', dest='id')
    return result.text

def translate_excel_column(input_file, output_file, sheet_name, column_name):
    df = pd.read_excel(input_file, sheet_name=sheet_name)
    df[column_name] = df[column_name].apply(translate_to_indonesian)

    with pd.ExcelWriter(output_file) as writer:
        df.to_excel(writer, index=False, sheet_name=sheet_name)
    print("Translation complete. Translated data saved to a new Excel file!")

# Example usage
input_file = '/content/drive/MyDrive/Portfolio/Sentiment Analysis User Reviews App GetContact/LabellingData GetContact/GetContact_DataTrainEngLabeledBalanced.xlsx'
output_file = '/content/drive/MyDrive/Portfolio/Sentiment Analysis User Reviews App GetContact/LabellingData GetContact/GetContact_DataTrainIdnLabeledBalanced.xlsx'
sheet_name = 'Sheet1'  # Change to your sheet name
column_name = 'content_normalisasi2'  # Change to the name of the column you want to translate

translate_excel_column(input_file, output_file, sheet_name, column_name)

Translation complete. Translated data saved to a new Excel file!


In [67]:
import pandas as pd
from googletrans import Translator

def translate_to_indonesian(text):
    translator = Translator()
    result = translator.translate(text, src='en', dest='id')
    return result.text

def translate_excel_column(input_file, output_file, sheet_name, column_name):
    df = pd.read_excel(input_file, sheet_name=sheet_name)
    df[column_name] = df[column_name].apply(translate_to_indonesian)

    with pd.ExcelWriter(output_file) as writer:
        df.to_excel(writer, index=False, sheet_name=sheet_name)
    print("Translation complete. Translated data saved to a new Excel file!")

# Example usage
input_file = '/content/drive/MyDrive/Portfolio/Sentiment Analysis User Reviews App GetContact/LabellingData GetContact/GetContact_DataTestEnglish.xlsx'
output_file = '/content/drive/MyDrive/Portfolio/Sentiment Analysis User Reviews App GetContact/LabellingData GetContact/GetContact_DataTestIndo.xlsx'
sheet_name = 'Sheet1'  # Change to your sheet name
column_name = 'content_normalisasi2'  # Change to the name of the column you want to translate

translate_excel_column(input_file, output_file, sheet_name, column_name)

Translation complete. Translated data saved to a new Excel file!
